This notebook is about an algorithm that can find the n-th regular number in linear time O(n). According to [Wikipedia](https://en.wikipedia.org/wiki/Regular_number#Number_theory), a regular number is formally defined as a number of the form $2^i \cdot 3^j \cdot 5^k$ where $i,j$ and $k$ are nonnegative integers. What make this problem interesting is that a similar problem was asked during Google interview, as reported in [this post on Stackoverflow](https://stackoverflow.com/questions/5505894/tricky-google-interview-question/5506635).

[In another post on  Stackoverflow](https://stackoverflow.com/questions/4600048/n%e1%b5%97%ca%b0-ugly-number), many solutions to find the n-th regular number are proposed. One among them is my own solution written in Python (here is the [link to my solution](https://stackoverflow.com/a/9007029/718529)).

# Outline

From this point on, I will discuss my solution. In section [The Algorithm](#The-Algorithm), the code of my algorithm is given. Next, instead of writting a mathematical proof to show that my algorithm is correct, I will verify it by comparing
the algorithm's generated sequence to datasets of regular number: 
- In section [Test Against Wikipedia's Data](#Test-Against-Wikipedia's-Data), a small dataset of size 26 is drawn from Wikipedia page 
- In section [Test Against Synthetic Data](#Test-Against-Synthetic-Data), a dataset of regular numbers less than or equal to 1000 is generated by brute force method


# The Algorithm

Below code will produce a sequence of regular number in ascending order. Starting from the first iteration, the algorithm takes O(1) to ouput `num=1` via `yield num` and O(1) for appending 1\*2, 1\*3, and 1\*5 to the three FIFO queues `qs`. Likewise, in later iteration (in `while` loop), it takes O(1) to (1) retrieve the next regular number, (2) output `num` and (3) append `num*2, num*3, num*5` to `qs`. Because of this, the total runtime from iteration 1 to iteration n will be O(n). When we run through iteration 1 to iteration n, the algorithm will generate a sequence of the first, the second, and so on up to the n-th regular numbers.


More detailed explanation as well as some illustrations can be found in:
- The section [Breif Explanation About the Algorithm](#Breif-Explanation-About-the-Algorithm) and,
- The sub-section [Example](#Example).

In [1]:
from collections import deque
bases = [2,3,5]

def regNumGenerator():
    qs = [deque([]) for _ in bases]
    counter = 1
    num = 1
    
    yield num
    print("Iteration {}".format(counter))
    print("num={}\n".format(num))
    [q.append(num*b) for q, b in zip(qs, bases)]
    
    counter+= 1
    
    while True:
        
        print("Iteration {}".format(counter))
        print("Queues Before popping: {}".format(qs))
        
        # (1) Retrieve the next regular number `num` by popping the smallest number from heads of the three queues
        num = min([q[0] for q in qs])
        [q.popleft() for q in qs if q[0] == num]
        
        print("num={}".format(num))
        print("Queues After poping: {}\n".format(qs))
        
        # (2) output the current regular number `num`
        yield num
        # (3) Add new members to tails of the three queues
        [q.append(num*b) for q, b in zip(qs, bases)]
        
        counter += 1
    

# Test Against Wikipedia's Data

On [the Wikipedia page on regular number](https://en.wikipedia.org/wiki/Regular_number#Number_theory), a sequence of regular numbers up to 60 is given.
> The first few regular numbers are <br>
1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 15, 16, 18, 20, 24, 25, 27, 30, 32, 36, 40, 45, 48, 50, 54, 60

In [2]:
wikidata = [1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 15, 16, 18, 20, 24, 25, 27, 30, 32, 36, 40, 45, 48, 50, 54, 60]

In [3]:
it = regNumGenerator()
my_generated_seq = []
for num in it:
    if num > 60:
        break
    else: my_generated_seq.append(num)

Iteration 1
num=1

Iteration 2
Queues Before popping: [deque([2]), deque([3]), deque([5])]
num=2
Queues After poping: [deque([]), deque([3]), deque([5])]

Iteration 3
Queues Before popping: [deque([4]), deque([3, 6]), deque([5, 10])]
num=3
Queues After poping: [deque([4]), deque([6]), deque([5, 10])]

Iteration 4
Queues Before popping: [deque([4, 6]), deque([6, 9]), deque([5, 10, 15])]
num=4
Queues After poping: [deque([6]), deque([6, 9]), deque([5, 10, 15])]

Iteration 5
Queues Before popping: [deque([6, 8]), deque([6, 9, 12]), deque([5, 10, 15, 20])]
num=5
Queues After poping: [deque([6, 8]), deque([6, 9, 12]), deque([10, 15, 20])]

Iteration 6
Queues Before popping: [deque([6, 8, 10]), deque([6, 9, 12, 15]), deque([10, 15, 20, 25])]
num=6
Queues After poping: [deque([8, 10]), deque([9, 12, 15]), deque([10, 15, 20, 25])]

Iteration 7
Queues Before popping: [deque([8, 10, 12]), deque([9, 12, 15, 18]), deque([10, 15, 20, 25, 30])]
num=8
Queues After poping: [deque([10, 12]), deque([9, 

In [4]:
#Verify by comparing against data from Wiki
assert my_generated_seq == wikidata
print(my_generated_seq)
print(wikidata)

[1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 15, 16, 18, 20, 24, 25, 27, 30, 32, 36, 40, 45, 48, 50, 54, 60]
[1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 15, 16, 18, 20, 24, 25, 27, 30, 32, 36, 40, 45, 48, 50, 54, 60]


# Breif Explanation About the Algorithm
```text
Iteration 1:
    output 1
    add new member (2, 3, 5) to the queue structure
    
Iteration 2:
    Pop minimum number from the queue structure which is 2
    output 2
    add new members (2*2, 2*3 and 2*5) to the queue structure
                            ...

Iteration i-th:
    Pop minimum number from the queue structure. Calling it `num`
    output `num`
    add new members (num*2, num*3 and num*5) to the queue structure
```

The queue structure consists of three sub queues.

In each iteration of the algorithm, there are two stages:
1. We know that the first regular number is 1. Suppose that the algorithm know that the i-th regular number is `num`. Then, `num` will be an output of the algorithm in iteration i. Near the end of the i-th iteration, the queue structure will be updated . This is done by adding `num*2` to the tail of the first sub queue, adding `num*3` the the tail of the second sub queue, and adding `num*5` to the tail of the third sub queue. 
2. From iteration 2 onwards, the queue structure will be non-empty when we enter the iteration. So, at the beginning of the iteration i (i$\geq$2), we can retrieve the i-th regular number from the queue structure. The algorithm does this by popping the least number from heads of the three sub queue. This newly discovered number will be called `num`, and it is used in stage 1.


## Example

Let have a look at the state of the queue structure inside iteration 9 and 10 (the following text is taken from the long output in previous section) :
```
Iteration 9
Queues Before popping: [deque([10, 12, 16, 18]), deque([12, 15, 18, 24, 27]), deque([10, 15, 20, 25, 30, 40, 45])]
num=10
Queues After poping: [deque([12, 16, 18]), deque([12, 15, 18, 24, 27]), deque([15, 20, 25, 30, 40, 45])]

Iteration 10
Queues Before popping: [deque([12, 16, 18, 20]), deque([12, 15, 18, 24, 27, 30]), deque([15, 20, 25, 30, 40, 45, 50])]
num=12
```
Inside iteration 9, num=10 is retrieved from heads of the three sub queues.
![Popping ten](regularNum01.png)

Afterwards, the algorithm output num=10 as the 9-th regular number.<br> 
Next, before the transition from iteration 9 to iteration 10 occurs, the algorithm adds three numbers to tails of three sub queus: 20 (10\*2), 30 (10\*3) and 50 (10\*5).
![Adding ten times bases](regularNum02.png)

# Test Against Synthetic Data



In below brute force method, regular numbers up to a 1000 are generated by constructing numbers of the form $2^i \cdot 3^j \cdot 5^k$. This can be done by first constucting three sequences: a sequence of $2^i$, a sequence of $3^j$ and a sequence of $5^k$. Then, pick three numbers, one from each of these sequences, and multiply them together.

When constructing each of the three sequences we can use 1000 as a cutoff. This is because, if a number is greater than 1000, then the result will definitely exceed 1000 when multiplying it with any other two numbers (Remember that we want a regular number less than or equal to 1000.) For example, $2^{10}=1024$ shouldn't be in a sequence of $2^i$ because $2^{10} \cdot 3^j \cdot 5^k$ exceeds 1000 no matter what $3^j$ and $5^k$ are.

Three sequences are stored in dictionary `power_dict`.

In [5]:
#We're going to generate a sequence of regular numbers up to 1000 by brute force method.
cutoff = 1000

def sequenceOfPower(base):
    e = 0
    while base**e <= cutoff:
        yield base**e
        e += 1

power_dict = {str(b)+'^{}'.format(n): list(sequenceOfPower(b)) for b, n in zip(bases,'ijk')}
print('power_dict =', power_dict)

from itertools import product
brute_force_data = sorted([x*y*z for x,y,z in product(*power_dict.values()) if x*y*z <= 1000])

power_dict = {'2^i': [1, 2, 4, 8, 16, 32, 64, 128, 256, 512], '3^j': [1, 3, 9, 27, 81, 243, 729], '5^k': [1, 5, 25, 125, 625]}


In [6]:
import os, sys

# Credit:
# This class is taken from https://stackoverflow.com/a/45669280/718529
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
        
with HiddenPrints():
    my_algo_seq = []
    for x in regNumGenerator():
        if x > 1000:
            break
        my_algo_seq.append(x)

In [7]:
import pandas as pd
df = pd.DataFrame(zip(brute_force_data, my_algo_seq), columns=['Brute Force', 'My Algo'])

assert df['Brute Force'].equals(df['My Algo'])
pd.set_option('display.max_rows', df.shape[0]+1)
print(df)

    Brute Force  My Algo
0             1        1
1             2        2
2             3        3
3             4        4
4             5        5
5             6        6
6             8        8
7             9        9
8            10       10
9            12       12
10           15       15
11           16       16
12           18       18
13           20       20
14           24       24
15           25       25
16           27       27
17           30       30
18           32       32
19           36       36
20           40       40
21           45       45
22           48       48
23           50       50
24           54       54
25           60       60
26           64       64
27           72       72
28           75       75
29           80       80
30           81       81
31           90       90
32           96       96
33          100      100
34          108      108
35          120      120
36          125      125
37          128      128
38          135      135
